# SoPra Team 17: Stats

In [ ]:
from github import Github
import requests
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt

token = ""
g = Github(token)
org = g.get_organization("SoPra-Team-17")
repos = org.get_repos()

print("Got {} repos".format(len(list(repos))))

## Pull Requests

In [ ]:
pulls = list()
for repo in repos:
    pulls += repo.get_pulls(state="all")

print("In total, {} pull requests have been submitted.".format(len(pulls)))

In [ ]:
merged = list()
for pull in pulls:
    if pull.merged:
        merged.append(pull)
        
print("Out of those, {} pull requests were merged:".format(len(merged)))

In [ ]:
labels = ["Merged", "Closed"]
numbers = [len(merged), len(pulls)-len(merged)]
fig1, ax1 = plt.subplots()
ax1.pie(numbers, labels=labels, startangle=90)
ax1.axis("equal")
plt.show()

In [ ]:
authors = {}
for pull in pulls:
    name = pull.user.name
    if name is None:
        name = pull.user.login
    cnt = authors.get(name, 0)
    authors[name] = cnt + 1

fig1, ax1 = plt.subplots()
ax1.set_title("Number of PRs by author")
ax1.pie(authors.values(), labels=authors.keys(), startangle=90, )
ax1.axis("equal")
plt.show()

### Reviews

In [ ]:
reviews = list()
for pull in pulls:
    reviews += pull.get_reviews()
    
print("Got {} reviews".format(len(reviews)))

In [ ]:
reviewsByUser = {}
for review in reviews:
    name = review.user.name
    if name is None:
        name = review.user.login
    cnt = reviewsByUser.get(name, 0)
    reviewsByUser[name] = cnt + 1

fig1, ax1 = plt.subplots()
ax1.set_title("Number of reviews by author")
ax1.pie(reviewsByUser.values(), labels=reviewsByUser.keys(), startangle=90)
ax1.axis("equal")
plt.show()

## Commits

In [ ]:
commitsByRepo = {}
authors = set()
for repo in repos:
    commitsByRepo[repo.name] = {}
    stats = repo.get_stats_contributors()
    if stats is None:
        continue
    for stat in stats:
        name = stat.author.name
        if name is None:
            name = stat.author.login
        authors.add(name)
        
        cnt = commitsByRepo[repo.name].get(name, 0)
        commitsByRepo[repo.name][name] = cnt + stat.total

In [ ]:
repoNames = commitsByRepo.keys()

contribData = [[0 for x in range(len(repoNames))] for y in range(len(authors))] 

for repoIndex, repoName in enumerate(repoNames):
    for authorIndex, authorName in enumerate(authors):
        contribData[authorIndex][repoIndex] = commitsByRepo[repoName].get(authorName, 0)
        
fig, ax = plt.subplots()
ax.set_title("Number of commits by author and repo")
im = ax.imshow(contribData)

# We want to show all ticks...
ax.set_xticks(np.arange(len(repoNames)))
ax.set_yticks(np.arange(len(authors)))
# ... and label them with the respective list entries
ax.set_xticklabels(repoNames)
ax.set_yticklabels(authors)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(authors)):
    for j in range(len(repoNames)):
        text = ax.text(j, i, contribData[i][j],
                       ha="center", va="center", color="w")

plt.show()

## CI

In [ ]:
total_ci_runs = 0
successful_ci_runs = 0

for repo in repos:
    query_url = f"https://api.github.com/repos/{org.login}/{repo.name}/actions/runs"
    params = {
        "status": "success"
    }
    headers = {'Authorization': f'token {token}'}

    result_all = requests.get(query_url, headers=headers).json()
    result_success = requests.get(query_url, headers=headers, params=params).json()
    
    total_ci_runs += result_all["total_count"]
    successful_ci_runs += result_success["total_count"]
    

print(f"{total_ci_runs} CI jobs were run in total, {successful_ci_runs} were successful!")

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title("CI Job results")
ax1.pie([successful_ci_runs, total_ci_runs-successful_ci_runs], labels=["Success", "Fail"], startangle=90)
ax1.axis("equal")
plt.show()